In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/commonlitreadabilityprize/sample_submission.csv
/kaggle/input/commonlitreadabilityprize/train.csv
/kaggle/input/commonlitreadabilityprize/test.csv


# CommonLit Readability Prize

Competition URL: https://www.kaggle.com/c/commonlitreadabilityprize/overview

**code credits**\
model training: https://www.kaggle.com/maunish/clrp-pytorch-roberta-finetune \
pooling method: https://www.kaggle.com/rhtsingh/utilizing-transformer-representations-efficiently \
feature build: https://www.kaggle.com/maximkazantsev/commonlit-readability-univ-sent-encoder-xgb \
model saving for future use: https://www.kaggle.com/questions-and-answers/92749 \
other: https://www.kaggle.com/vineethakkinapalli/clrp-roberta-base-representation-techniques

In this notebook I try to combine features extracted from excerpt into the model training process. There are 3 basic model I used, RoBertA with last 4 layers representations, RoBertA with a attention head, and Attention Pooling method on RoBertA.
The submission answer is the average value of all 15 models (each model has five submodels).

In [27]:
!pip3 install accelerate

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import re
import seaborn as sns
from transformers import AutoModel,AutoConfig,AutoTokenizer,get_cosine_schedule_with_warmup
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from accelerate import Accelerator
import random
import os
import numpy as np
import torch
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
import joblib

from colorama import Fore, Back, Style
r_ = Fore.RED
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
y_ = Fore.YELLOW
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL

In [29]:
train = pd.read_csv("/kaggle/input/commonlitreadabilityprize/train.csv")
test = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")
sample = pd.read_csv("/kaggle/input/commonlitreadabilityprize/sample_submission.csv")

In [30]:
#train data excerpt length
train["len"] = train.excerpt.apply(len)
test["len"] = test.excerpt.apply(len)

In [31]:
train["excerpt"] = train.excerpt.apply(lambda x: x.replace("\n"," "))
test["excerpt"] = test.excerpt.apply(lambda x: x.replace("\n"," "))

In [32]:
def feature(excerpt, stop_words):
    dialogue_counts=excerpt.count('"')/2
    num_sentence = len(sent_tokenize(excerpt))
    
    words = word_tokenize(re.sub("[^a-zA-Z]", " ", excerpt).lower())
    initial_num_words = len(words)
    target_words = [word for word in words if word not in stop_words]
    processed_num_words=len(target_words)
    text_shrinkage = processed_num_words/initial_num_words
    avg_sentence_length = initial_num_words / num_sentence
    
    return dialogue_counts, num_sentence, processed_num_words, text_shrinkage, avg_sentence_length

In [33]:
stop_words = set(stopwords.words("english"))
train['feature'] = train.excerpt.apply(feature, stop_words= stop_words)
test['feature'] = test.excerpt.apply(feature, stop_words= stop_words)
train["dialogue"], train["num_sentence"], train["num_processed_words"], train['text_shrinkage'], \
train["avg_sentence_length"] = zip(*train.feature)

test["dialogue"], test["num_sentence"], test["num_processed_words"], test['text_shrinkage'], \
test["avg_sentence_length"] = zip(*test.feature)

In [34]:
train.drop(columns="feature", inplace=True)
test.drop(columns="feature", inplace=True)

In [35]:
class CLRPDataset(Dataset):
    def __init__(self,df,tokenizer,feature,max_len=256, train=True):
        self.excerpt = df['excerpt'].to_numpy()
        self.train = train
        if self.train:
            self.targets = df['target'].to_numpy()
        self.feature = feature
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        
        feature = torch.tensor(self.feature[idx],dtype=torch.float,)
        
        if self.train:
            target = torch.tensor(self.targets[idx],dtype=torch.float) 
            return encode, feature, target
        else:
            return encode, feature
    
    def __len__(self):
        return len(self.excerpt)

In [36]:
config = {
    'lr': 2e-5,
    'wd':0.01,
    'batch_size':16,
    'valid_step':10,
    'max_len':500,
    'epochs':5,
    'nfolds':5,
    'seed':9527,
    'model_path':'roberta-base',
}

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(seed=config['seed'])

In [37]:
for i in range(config['nfolds']):
    os.makedirs(f'concat_model{i}',exist_ok=True)
for i in range(config['nfolds']):
    os.makedirs(f'atHead_model{i}',exist_ok=True)
for i in range(config['nfolds']):
    os.makedirs(f'atPool_model{i}',exist_ok=True)
os.makedirs('scaler',exist_ok=True)

In [38]:
num_bins = int(np.floor(1 + np.log2(len(train))))
train.loc[:,'bins'] = pd.cut(train['target'],bins=num_bins,labels=False)
bins = train.bins.to_numpy()

train['Fold'] = -1
kfold = StratifiedKFold(n_splits=config['nfolds'],shuffle=True,random_state=config['seed'])
for k , (train_idx,valid_idx) in enumerate(kfold.split(X=train, y=bins)):
    train.loc[valid_idx,'Fold'] = k

In [39]:
class Model(nn.Module):
    def __init__(self,path):
        super(Model,self).__init__()
        self.config = AutoConfig.from_pretrained(path)
        self.config.update({'output_hidden_states':True,"hidden_dropout_prob": 0.0})
        self.roberta = AutoModel.from_pretrained(path,config=self.config)
        self.linear = nn.Linear(self.config.hidden_size*4+5, 1, 1)

    def forward(self,feature, **xb):
        x = self.roberta(**xb)
        x = torch.stack(x[2])
        x = torch.cat((x[-1], x[-2], x[-3], x[-4]),-1)
        x = x[:, 0]
        x = torch.cat((x,feature), -1)
        x = self.linear(x)
        return x

In [40]:
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim):
        super(AttentionHead, self).__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim
        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim

    def forward(self, features):
        att = torch.tanh(self.W(features))
        score = self.V(att)
        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector
    
class ATHeadModel(nn.Module):
    def __init__(self,path):
        super(ATHeadModel,self).__init__()
        self.roberta = AutoModel.from_pretrained(path)  
        self.config = AutoConfig.from_pretrained(path)
        self.head = AttentionHead(self.config.hidden_size,self.config.hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(self.config.hidden_size+5,1)

    def forward(self,feature,**xb):
        x = self.roberta(**xb)[0]
        x = self.head(x)
        x = self.dropout(x)
        x = torch.cat((x,feature), -1)
        x = self.linear(x)
        return x

In [41]:
class AttentionPoolingModel(nn.Module):
    def __init__(self, path):
        super(AttentionPoolingModel, self).__init__() 
        self.config = AutoConfig.from_pretrained(path)
        self.config.update({'output_hidden_states':True,"hidden_dropout_prob": 0.0})
        self.roberta = AutoModel.from_pretrained(path,config=self.config)
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(self.config.hidden_size+5,1)

        q_t = np.random.normal(loc=0.0, scale=0.1, size=(1, self.config.hidden_size))
        self.q = nn.Parameter(torch.from_numpy(q_t).float())
        w_ht = np.random.normal(loc=0.0, scale=0.1, size=(self.config.hidden_size, self.config.hidden_size))
        self.w_h = nn.Parameter(torch.from_numpy(w_ht).float())

    def forward(self, feature, **xb):
        x = self.roberta(**xb)
        x = torch.stack(x[2])
        x = torch.stack([x[layer_i][:, 0].squeeze() for layer_i in range(1, self.config.num_hidden_layers+1)], dim=-1)
        x = x.view(-1, self.config.num_hidden_layers, self.config.hidden_size)
        x = self.attention(x)
        x = self.dropout(x)
        x = torch.cat((x,feature), -1)
        x = self.linear(x)
        return x

    def attention(self, h):
        v = torch.matmul(self.q, h.transpose(-2, -1)).squeeze(1)
        v = F.softmax(v, -1)
        v_temp = torch.matmul(v.unsqueeze(1), h).transpose(-2, -1)
        v = torch.matmul(self.w_h.transpose(1, 0), v_temp).squeeze(2)
        return v

In [42]:
def run(fold, MODEL, verbose=True):
    
    
    model_dict = {"concat_model":Model,
                  "atHead_model":ATHeadModel,
                  "atPool_model":AttentionPoolingModel}
    
    def loss_fn(outputs,targets):
        outputs = outputs.view(-1)
        targets = targets.view(-1)
        return torch.sqrt(nn.MSELoss()(outputs,targets))
    
    def train_and_evaluate_loop(train_loader,valid_loader,model,loss_fn,optimizer,epoch,fold,best_loss,valid_step=10,lr_scheduler=None):
        train_loss = 0
        for i, (inputs1,feature1,targets1) in enumerate(train_loader):
            model.train()
            optimizer.zero_grad()
            inputs1 = {key:val.reshape(val.shape[0],-1) for key,val in inputs1.items()}
            outputs1 = model(feature1, **inputs1)
            loss1 = loss_fn(outputs1,targets1)
            loss1.backward()
            optimizer.step()
            
            train_loss += loss1.item()
            
            if lr_scheduler:
                lr_scheduler.step()
            
            #evaluating for every valid_step
            #if (i % valid_step == 0) or ((i + 1) == len(train_loader)):
            if (i + 1) == len(train_loader):
                model.eval()
                valid_loss = 0
                with torch.no_grad():
                    for j, (inputs2,feature2,targets2) in enumerate(valid_loader):
                        inputs2 = {key:val.reshape(val.shape[0],-1) for key,val in inputs2.items()}
                        outputs2 = model(feature2,**inputs2)
                        loss2 = loss_fn(outputs2,targets2)
                        valid_loss += loss2.item()
                     
                    valid_loss /= len(valid_loader)
                    if valid_loss <= best_loss:
                        if verbose:
                            print(f"epoch:{epoch} | Train Loss:{train_loss/(i+1)} | Validation loss:{valid_loss}")
                            print(f"{g_}Validation loss Decreased from {best_loss} to {valid_loss}{sr_}")

                        best_loss = valid_loss
                        torch.save(model.state_dict(),f'./{MODEL}{fold}/model.bin')
                        tokenizer.save_pretrained(f'./{MODEL}{fold}')
                        
        return best_loss
    
    accelerator = Accelerator()
    print(f"{accelerator.device} is used")
    
    x_train,x_valid = train.query(f"Fold != {fold}"),train.query(f"Fold == {fold}")
    
    if os.path.isfile(f"scaler/scaler_fold{fold}.gz"):
        scaler = joblib.load(f"scaler/scaler_fold{fold}.gz") 
    else:
        scaler = StandardScaler()
        scaler.fit(x_train.iloc[:,9:].to_numpy())
        joblib.dump(scaler, f"scaler/scaler_fold{fold}.gz")
    x_train_feature=scaler.transform(x_train.iloc[:,9:].to_numpy())
    x_valid_feature=scaler.transform(x_valid.iloc[:,9:].to_numpy())
    
    
    tokenizer = AutoTokenizer.from_pretrained(config['model_path'])
    model = model_dict[MODEL](config['model_path'])

    train_ds = CLRPDataset(x_train,tokenizer,x_train_feature,config['max_len'])
    train_dl = DataLoader(train_ds,
                        batch_size = config["batch_size"],
                        shuffle=True,
                        num_workers = 4,
                        pin_memory=True,
                        drop_last=False)

    valid_ds = CLRPDataset(x_valid,tokenizer,x_valid_feature,config['max_len'])
    valid_dl = DataLoader(valid_ds,
                        batch_size = config["batch_size"],
                        shuffle=False,
                        num_workers = 4,
                        pin_memory=True,
                        drop_last=False)

    optimizer = optim.AdamW(model.parameters(),lr=config['lr'],weight_decay=config['wd'])
    lr_scheduler = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps= 10 * len(train_dl))

    model,train_dl,valid_dl,optimizer,lr_scheduler = accelerator.prepare(model,train_dl,valid_dl,optimizer,lr_scheduler)

    print(f"Fold: {fold}")
    best_loss = 9999
    for epoch in range(config["epochs"]):
        print(f"Epoch Started:{epoch}")
        best_loss = train_and_evaluate_loop(train_dl,valid_dl,model,loss_fn,
                                            optimizer,epoch,fold,best_loss,
                                            valid_step=config['valid_step'],lr_scheduler=lr_scheduler)

In [43]:
def get_prediction(df,Fold,MODEL,path,model_path,device='cuda'):
    outputs = np.zeros(len(df))
    for f in range(Fold):
        accelerator = Accelerator()
        model = MODEL(model_path)
        model.load_state_dict(torch.load(path+f"{f}/model.bin",map_location=device))
        model.eval()
        

        tokenizer = AutoTokenizer.from_pretrained(model_path)
        
        scaler = joblib.load(f"scaler/scaler_fold{f}.gz")
        df_feature = scaler.transform(df.iloc[:,5:].to_numpy())

        test_ds = CLRPDataset(df,tokenizer,df_feature, config['max_len'],train=False)
        test_dl = DataLoader(test_ds,
                            batch_size = config["batch_size"],
                            shuffle=False,
                            num_workers = 4,
                            pin_memory=True)
        
        model,test_dl = accelerator.prepare(model,test_dl)

        predictions = list()
        for i, (inputs, feature) in enumerate(test_dl):
            inputs = {key:val.reshape(val.shape[0],-1) for key,val in inputs.items()}
            outputs = model(feature, **inputs)
            outputs = outputs.cpu().detach().numpy().ravel().tolist()
            predictions.extend(outputs)

        torch.cuda.empty_cache()
        outputs+=np.array(predictions)
    return outputs/Fold

In [ ]:
for f in range(config['nfolds']):
    run(f, 'concat_model')

cuda is used


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Fold: 1
Epoch Started:0
epoch:0 | Train Loss:0.6750002757344448 | Validation loss:0.49220481763283414
Validation loss Decreased from 9999 to 0.49220481763283414
Epoch Started:1
epoch:1 | Train Loss:0.4402375724953665 | Validation loss:0.4789656044708358
Validation loss Decreased from 0.49220481763283414 to 0.4789656044708358
Epoch Started:2
Epoch Started:3
Epoch Started:4
cuda is used


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Fold: 2
Epoch Started:0
epoch:0 | Train Loss:0.7102370883377505 | Validation loss:0.5416221668322881
Validation loss Decreased from 9999 to 0.5416221668322881
Epoch Started:1
epoch:1 | Train Loss:0.45342342471572716 | Validation loss:0.5234515708353784
Validation loss Decreased from 0.5416221668322881 to 0.5234515708353784
Epoch Started:2
Epoch Started:3
epoch:3 | Train Loss:0.24119354165355925 | Validation loss:0.4954050588938925
Validation loss Decreased from 0.5234515708353784 to 0.4954050588938925
Epoch Started:4
cuda is used


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Fold: 3
Epoch Started:0


In [ ]:
for f in range(config['nfolds']):
    run(f, 'atHead_model')

In [ ]:
for f in range(config['nfolds']):
    run(f, 'atPool_model')

In [ ]:
answer1=get_prediction(test,config["nfolds"],Model,"concat_model",config['model_path'],device='cuda')
answer2=get_prediction(test,config["nfolds"],ATHeadModel,"atHead_model",config['model_path'],device='cuda')
answer3=get_prediction(test,config["nfolds"],AttentionPoolingModel,"atPool_model",config['model_path'],device='cuda')

In [ ]:
answer=(answer1+answer2+answer3)/3
sample['target']=answer
sample.to_csv('submission.csv',index=False)